# <b>Price Predictor

#### First we'll do a week of each company

In [111]:
import numpy as np
import pandas as pd
import matplotlib
import yfinance as yf

In [112]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S%26P_500_component_stocks')
table_symbol = data[0]
table_industry = data[0]['GICS Sector']

#these 2 lists are in order with one another
symbols = list(table_symbol.Symbol.values)
industries = list(table_industry.values) ##Industry Sector


remove = []

symbols_to_remove = ['BF.B', 'BRK.B', 'KVUE', 'VLTO','SPY']

for i, symbol in enumerate(symbols):
    if symbol in symbols_to_remove:
        remove.append(i)
#print(remove)

#list(set(industries))

for i in remove:
    symbols.remove(symbols[i])
    industries.remove(industries[i])

print(len(symbols))

symbol_industry = dict(zip(symbols,industries))
print(len(symbol_industry))

499
499


In [113]:
tickers = yf.Tickers('AAPL')

APPL = tickers.tickers['AAPL'].history(period="5d", start="2022-01-01", end="2022-12-31")
APPL.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,175.821115,180.814070,175.702476,179.953888,104487900,0.0,0.0
2022-01-04 00:00:00-05:00,180.566907,180.873403,177.096548,177.669998,99310400,0.0,0.0
2022-01-05 00:00:00-05:00,177.581006,178.134677,172.667149,172.943985,94537600,0.0,0.0
2022-01-06 00:00:00-05:00,170.749066,173.319700,169.701043,170.056976,96904000,0.0,0.0
2022-01-07 00:00:00-05:00,170.936935,172.172814,169.097946,170.225067,86709100,0.0,0.0
2022-01-10 00:00:00-05:00,167.169944,170.551308,166.270221,170.244812,106765600,0.0,0.0
2022-01-11 00:00:00-05:00,170.373387,173.201065,168.890332,173.102203,76138300,0.0,0.0
2022-01-12 00:00:00-05:00,174.130420,175.178443,172.845118,173.547089,74805200,0.0,0.0
2022-01-13 00:00:00-05:00,173.794253,174.624760,169.849322,170.244812,84505800,0.0,0.0


In [114]:
drop_cols = ['Open', 'Low', 'High','Volume', 'Dividends', 'Stock Splits']

In [116]:
APPL = APPL.drop(columns=drop_cols, axis = 1)
APPL.index = APPL.index.astype(str).str.split(' ').str[0]
APPL

,Close
Date,
2022-01-03,179.953888
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056976
2022-01-07,170.225067
...,...
2022-12-23,131.127060
2022-12-27,129.307220
2022-12-28,125.339409


In [117]:
APPL.loc["2022-06-15"]

Close    134.26767
Name: 2022-06-15, dtype: float64

In [119]:
APPL.loc["2022-11-25"]
#APPL.loc["2022-11-24"]

Close    147.286743
Name: 2022-11-25, dtype: float64

In [120]:
holidays = ["2022-01-17","2022-02-21","2022-04-15", "2022-05-30", "2022-06-20", "2022-07-4", "2022-09-5", "2022-11-24","2022-12-26"]
len(holidays)

9

In [128]:
holiday_df = pd.DataFrame(index=holidays)
holiday_df['Date'] = holiday_df.index

# Add another column 'Close' with NaN values
holiday_df['Close'] = float('nan')

# Set the 'Date' column as the index
holiday_df.set_index('Date', inplace=True)

# Print the resulting DataFrame
holiday_df

,Close
Date,
2022-01-17,NaN
2022-02-21,NaN
2022-04-15,NaN
2022-05-30,NaN
2022-06-20,NaN
2022-07-4,NaN
2022-09-5,NaN
2022-11-24,NaN
2022-12-26,NaN


In [130]:
# Concatenate the original DataFrame with the holiday DataFrame
frames = [APPL, holiday_df]
result_df = pd.concat(frames)

# Sort the index to maintain order
result_df = result_df.sort_index()

# Print the resulting DataFrame
result_df

,Close
Date,
2022-01-03,179.953888
2022-01-04,177.669998
2022-01-05,172.943985
2022-01-06,170.056976
2022-01-07,170.225067
...,...
2022-12-26,NaN
2022-12-27,129.307220
2022-12-28,125.339409


In [ ]:
import pandas as pd
import yfinance as yf

# Transpose the DataFrame
APPL_transposed = APPL.T

# Sort the index (weeks) in chronological order
APPL_transposed = APPL_transposed.sort_index()

# Rename the index to 'APPL Week #X'
APPL_transposed.index = [f'APPL Week #{i}' for i in range(1, len(APPL_transposed) + 1)]

# Rename the columns to 'Day X'
APPL_transposed.columns = [f'Day {i}' for i in range(1, APPL_transposed.shape[1] + 1)]

# Display the resulting DataFrame
print(APPL_transposed)

In [ ]:
#data = yf.download("AAPL", period="5d")
#data

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # download data faster

# download dataframe
data1 = pdr.get_data_yahoo("JPM", start="2022-01-01", end="2022-12-31")

In [ ]:
data1

In [ ]:
drop_cols = ['Date', 'Volume', 'Open', 'Low', 'High', 'OpenInt']